In [22]:
from spacy.tokens import DocBin
import spacy
import json
from tqdm import tqdm
import random

# Name Entity Recognition (NER)¶
First i train a spacy model to recorgnize names of lectureres in a review

In [16]:
downloadModel = "python -m spacy download en_core_web_sm"
!{downloadModel}

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [17]:
def load_data(file):
    with open(file, "r") as f:
        data = json.load(f)
    return (data["annotations"])

I used the https://github.com/tecoholic/ner-annotator for annotation and the result is the annotiontion.json file

In [19]:
train_data = load_data("/kaggle/input/annotations-for-spacy-ner/Annotation/annotations.json")

In [20]:
# Remove None  values
train_data = [item for item in train_data if item is not None]

In [22]:
# Set entity
entity_name = "PERSON"
# Convert to turple
train_data = [tuple(i) for i in train_data]

In [23]:
for i in train_data:
    if not i[1]['entities']:
        i[1]['entities'] = [(0, 0, entity_name)]
    else:
        i[1]['entities'] = [tuple(entity) for entity in i[1]['entities']]

In [24]:
# Display train_data
train_data

[('Dr. Helen is an interesting lecturer whoŸ??s knowledgeable and has a solid expertise on her field of study. Her teaching style is also interactive, which gives room for students to ask questions and understand more in the classroom. Her communication skills are also remarkable as she explains her teachings in a detailed and simple manner. Classroom management is average because some students take advantage of the fact that sheŸ??s welcoming sometimes. Unfortunately, the accessments are also demanding because she suspects she has given given us all the information we need during teacg sessions. Dr. Helen is accessible to students and always ready to answer questions in areas which they find difficult. She has a strong sense of professionalism and ethics which makes her treat students with respect and fairness as long as they respect themselves too.',
  {'entities': [(4, 9, 'PERSON')]}),
 ('Dr. Helen Edogbanya demonstrates good knowledge and expertise in the subject matter. Her teachi

In [25]:
nlp = spacy.load("en_core_web_sm") # load spacy model

db = DocBin() # create a DocBin object

for text, annot in tqdm(train_data): # data in previous format
    doc = nlp.make_doc(text) # create doc object from text
    ents = []
    for start, end, label in annot["entities"]: # add character indexes
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents # label the text with the ents
    db.add(doc)

100%|██████████| 78/78 [00:00<00:00, 612.83it/s]

Skipping entity


In [26]:
# train_data 
train_data_path = "train.spacy"
db.to_disk(train_data_path)

In [27]:
config_path = '/kaggle/input/config-for-spacy-ner/config/config.cfg'
base_config_path = '/kaggle/input/config-for-spacy-ner/config/base_config.cfg'

command = "python -m spacy init fill-config {} {}"
formatted_command = command.format(base_config_path, config_path)

!{formatted_command}

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [ ]:
output_path = "model"

command = "python -m spacy train {} --output {} --paths.train {} --paths.dev {}"
formatted_command = command.format(config_path, output_path, train_data_path, train_data_path)

!{formatted_command}

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

# TEST Model

In [3]:
!pip install chardet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.4/199.4 kB 7.6 MB/s eta 0:00:00


In [5]:
import pandas as pd
import csv
import chardet

In [6]:
# Load model
nlp1 = spacy.load(r"model/model-best")

In [7]:
test_path = '/kaggle/input/lecturer-review-testset/Test.csv'  #

def read_csv_file(file_path):
    rows = []

    # Detect the encoding of the CSV file
    with open(file_path, 'rb') as raw_file:
        result = chardet.detect(raw_file.read())
        csv_encoding = result['encoding']

    # Read the CSV file with the detected encoding
    with open(file_path, 'r', encoding=csv_encoding) as csv_file:
        csv_reader = csv.reader(csv_file)
        rows = list(csv_reader)
    
    return rows

# Assuming 'file_path' is the path to the CSV file
tests = read_csv_file(test_path)

print(tests)


[['REVIEW'], ["Mr. Sebastine Emmanuel possesses good knowledge and expertise in the subject matter. His teaching style is effective, employing various strategies to engage students and cater to different learning preferences. However, there is room for improvement in communication skills to enhance clarity and foster deeper understanding. Classroom management is well-maintained, creating an atmosphere conducive to learning. Students appreciate Mr. Sebastine Emmanuel's fair and comprehensive assessments, which allow them to demonstrate their knowledge and skills. However, accessibility remains a challenge and should be addressed to ensure inclusivity for all students. It is important to emphasize professionalism and ethics in all aspects of teaching, which Mr. Sebastine Emmanuel should strive to uphold."], ["Mr. Umaru Sani is a knowledgeable and experienced educator who demonstrates a strong command of the subject matter. His teaching style is effective, utilizing a combination of tradi

In [8]:
def process_ner(reviews):
    docs = []
    # Skip the first row (reviews[0] is the header)
    for review in reviews[1:]:
        text = review[0]
        doc = nlp1(text) 
        docs.append(doc)
    return docs

docs = process_ner(tests)

In [9]:
# display in Jupyter
spacy.displacy.render(docs, style="ent", jupyter=True)

In [11]:
def extract_and_count_person_entities(documents):
    
    entities = []

    # Iterate over the entities in the documents
    for doc in documents:
        for entity in doc.ents:
            # Check if the entity label is 'PERSON'
            if entity.label_ == "PERSON":
                entities.append(entity.text.title())  # Convert to title case and append to entities
    return entities

entities = extract_and_count_person_entities(docs)  

In [13]:
# Create a DataFrame from the entities list
df = pd.DataFrame(entities, columns=["Entities"])

# Count the occurrences of each entity
entity_counts = df["Entities"].value_counts().reset_index()

# Rename the columns
entity_counts.columns = ["Entities", "Count"]

In [14]:
entity_counts

,Entities,Count
0,David Lanleige,5
1,Helen Edogbanya,5
2,Sebastine Emmanuel,3
3,Wakili Adamu,3
4,Ibe Ibenu,3
5,Umaru Sani,2
6,Veronica Cyril-Okeme,2
7,Jessica Gyegwe,2
8,Emmanuel Akaligwo,2
9,Luke Arinze Ozioko,2


# Aspect-Triplet Extraction
Next i perform **Aspect-Triplet Extraction** using the ***pyABSA** library* in other to extract Aspect, Opinion and Sentiment from each review

In [1]:
# install the library if not already insatlled
!pip install pyabsa

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.5/526.5 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.0/153.0 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 31.7 MB/s eta 0:00:0000:0100:01
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16181 sha256=f967dd1ffc6bde0c88f458e5b7d8a159191bc333f9e017136bd9291615257684
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval
  Attempting uninstall: scipy
    Found existing installation: scipy 1.11.1
    Uninstalling scipy-1.11.1:
      Successfully uninstalled scipy-1.11.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. Th

In [ ]:
#!pip install transformers==4.29.0

In [3]:
# import available checkpoints
from pyabsa import (
    TaskCodeOption,
    available_checkpoints,
)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 # import available checkpoints                                                               │
│ ❱ 2 from pyabsa import (                                                                         │
│   3 │   TaskCodeOption,                                                                          │
│   4 │   available_checkpoints,                                                                   │
│   5 )                                                                                            │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/pyabsa/__init__.py:13 in <module>                        │
│                                                                                                  │
│   10 __version__ = "2.3.3"                                                                       │
│   11                                                                                             │
│   12                                                                                             │
│ ❱ 13 from pyabsa.utils.notification_utils.notification_utils import (                            │
│   14 │   check_emergency_notification,                                                           │
│   15 )                                                                                           │
│   16                                                                                             │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/pyabsa/utils/__init__.py:9 in <module>                   │
│                                                                                                  │
│    6 # Copyright (C) 2021. All Rights Reserved.                                                  │
│    7                                                                                             │
│    8 from pyabsa.utils.data_utils.dataset_item import DatasetItem                                │
│ ❱  9 from pyabsa.utils.absa_utils.make_absa_dataset import make_ABSA_dataset                     │
│   10 from pyabsa.utils.absa_utils.absa_utils import (                                            │
│   11 │   generate_inference_set_for_apc,                                                         │
│   12 │   convert_apc_set_to_atepc_set,                                                           │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/pyabsa/utils/absa_utils/make_absa_dataset.py:18 in       │
│ <module>                                                                                         │
│                                                                                                  │
│    15                                                                                            │
│    16 from pyabsa.framework.flag_class.flag_template import LabelPaddingOption                   │
│    17                                                                                            │
│ ❱  18 from pyabsa.tasks.AspectTermExtraction.prediction.aspect_extractor import (                │
│    19 │   AspectExtractor,                                                                       │
│    20 )                                                                                          │
│    21 from pyabsa.utils.pyabsa_utils import fprint                                               │
│                                                            

In [ ]:
# choose task
task = TaskCodeOption.Aspect_Sentiment_Triplet_Extraction

In [ ]:
# select the check point and model
from pyabsa import AspectSentimentTripletExtraction as ASTE
triplet_extractor = ASTE.AspectSentimentTripletExtractor(checkpoint="multilingual")

In [ ]:
a = triplet_extractor.predict("customer service is great")

In [ ]:
a

In [ ]:
# Load dataset
review_path = "/kaggle/input/lecturer-reviewful/reviews.csv"

review_corpus = read_csv_file(review_path)

In [ ]:
review_corpus

In [ ]:
review_docs = process_ner(review_corpus)

In [ ]:
review_entities = extract_and_count_person_entities(review_docs) 

In [ ]:
review_entities

In [ ]:
# Create a DataFrame from the entities list
df1 = pd.DataFrame(review_entities, columns=["Entities"])

# Count the occurrences of each entity
review_entity_counts = df1["Entities"].value_counts().reset_index()

# Rename the columns
review_entity_counts.columns = ["Entities", "Count"]

In [ ]:
review_entity_counts

In [ ]:
# Function to split text into sentences
def split_into_sentences(text):
    sentences = [sentence.strip() for sentence in text.split('.') if sentence.strip()]
    sentences = [sentence for sublist in [sentence.split(',') for sentence in sentences] for sentence in sublist]
    return sentences


In [ ]:
# Redefined function to extract entity and aspect triplets
def extract_entity_and_triplets(review_corpus):
    results = []
    #review_entities = extract_and_count_person_entities(docs)  # Assuming you have 'docs' containing the processed reviews
    
    # Iterate over each review text in the corpus
    for review_text in review_corpus:
        sentences = split_into_sentences(review_text[0])
        entity = review_entities[len(results)]  # Get the entity for this review
        review_results = []
        # Predict triplets for each sentence and append the non-empty results
        for sentence in sentences:
            result = triplet_extractor.predict(sentence)
            if result['Triplets']:
                review_results.append(result)
        if review_results:
            results.append((entity, review_results))
            print(results)
    return results


In [ ]:
# Assuming 'review_corpus1' is the list of reviews you have
result = extract_entity_and_triplets(review_corpus)


In [ ]:
# Create a DataFrame from the results
def create_dataframe_from_results(results):
    entities_list = []
    aspects_list = []
    opinions_list = []
    polarity_list = []
    # Iterate over each result
    for entity, review in results:
        # Iterate over each dictionary in the element
        for sentence in review:
            # Extract the triplet values from the sentence where there is no error
            if not sentence.get('error'):
                aspect = sentence['Triplets'][0]['Aspect']
                opinion = sentence['Triplets'][0]['Opinion']
                polarity = sentence['Triplets'][0]['Polarity']
                # Append the values to the corresponding lists
                entities_list.append(entity)
                aspects_list.append(aspect)
                opinions_list.append(opinion)
                polarity_list.append(polarity)
    # Create a DataFrame from the extracted triplet values
    df = pd.DataFrame({
        'Entity': entities_list,
        'Aspect': aspects_list,
        'Opinion': opinions_list,
        'Polarity': polarity_list
    })
    return df
df_result1 = create_dataframe_from_results(result1)
